### Define the PyTorch model as it is in Tractolearn

In [1]:
import torch.nn as nn
from torch.nn import functional as F

class AETorch(nn.Module):
    """Strided convolution-upsampling-based AE using reflection-padding and
    increasing feature maps in decoder.
    """

    def __init__(self, latent_space_dims):
        super(AETorch, self).__init__()

        self.kernel_size = 3
        self.latent_space_dims = latent_space_dims

        self.pad = nn.ReflectionPad1d(1)

        def pre_pad(m):
            return nn.Sequential(self.pad, m)

        self.encod_conv1 = pre_pad(
            nn.Conv1d(3, 32, self.kernel_size, stride=2, padding=0)
        )
        self.encod_conv2 = pre_pad(
            nn.Conv1d(32, 64, self.kernel_size, stride=2, padding=0)
        )
        self.encod_conv3 = pre_pad(
            nn.Conv1d(64, 128, self.kernel_size, stride=2, padding=0)
        )
        self.encod_conv4 = pre_pad(
            nn.Conv1d(128, 256, self.kernel_size, stride=2, padding=0)
        )
        self.encod_conv5 = pre_pad(
            nn.Conv1d(256, 512, self.kernel_size, stride=2, padding=0)
        )
        self.encod_conv6 = pre_pad(
            nn.Conv1d(512, 1024, self.kernel_size, stride=1, padding=0)
        )

        self.fc1 = nn.Linear(8192, self.latent_space_dims)  # 8192 = 1024*8
        self.fc2 = nn.Linear(self.latent_space_dims, 8192)

        self.decod_conv1 = pre_pad(
            nn.Conv1d(1024, 512, self.kernel_size, stride=1, padding=0)
        )
        self.upsampl1 = nn.Upsample(
            scale_factor=2, mode="linear", align_corners=False
        )
        self.decod_conv2 = pre_pad(
            nn.Conv1d(512, 256, self.kernel_size, stride=1, padding=0)
        )
        self.upsampl2 = nn.Upsample(
            scale_factor=2, mode="linear", align_corners=False
        )
        self.decod_conv3 = pre_pad(
            nn.Conv1d(256, 128, self.kernel_size, stride=1, padding=0)
        )
        self.upsampl3 = nn.Upsample(
            scale_factor=2, mode="linear", align_corners=False
        )
        self.decod_conv4 = pre_pad(
            nn.Conv1d(128, 64, self.kernel_size, stride=1, padding=0)
        )
        self.upsampl4 = nn.Upsample(
            scale_factor=2, mode="linear", align_corners=False
        )
        self.decod_conv5 = pre_pad(
            nn.Conv1d(64, 32, self.kernel_size, stride=1, padding=0)
        )
        self.upsampl5 = nn.Upsample(
            scale_factor=2, mode="linear", align_corners=False
        )
        self.decod_conv6 = pre_pad(
            nn.Conv1d(32, 3, self.kernel_size, stride=1, padding=0)
        )
    def encode(self, x):
        h1 = F.relu(self.encod_conv1(x))
        h2 = F.relu(self.encod_conv2(h1))
        h3 = F.relu(self.encod_conv3(h2))
        h4 = F.relu(self.encod_conv4(h3))
        h5 = F.relu(self.encod_conv5(h4))
        h6 = self.encod_conv6(h5)

        self.encoder_out_size = (h6.shape[1], h6.shape[2])

        # Flatten
        h7 = h6.view(-1, self.encoder_out_size[0] * self.encoder_out_size[1])

        fc1 = self.fc1(h7)

        return fc1

    def decode(self, z):
        fc = self.fc2(z)
        fc_reshape = fc.view(
            -1, self.encoder_out_size[0], self.encoder_out_size[1]
        )
        h1 = F.relu(self.decod_conv1(fc_reshape))
        h2 = self.upsampl1(h1)
        h3 = F.relu(self.decod_conv2(h2))
        h4 = self.upsampl2(h3)
        h5 = F.relu(self.decod_conv3(h4))
        h6 = self.upsampl3(h5)
        h7 = F.relu(self.decod_conv4(h6))
        h8 = self.upsampl4(h7)
        h9 = F.relu(self.decod_conv5(h8))
        h10 = self.upsampl5(h9)
        h11 = self.decod_conv6(h10)

        return h11

    def forward(self, x):
        encoded = self.encode(x)
        return self.decode(encoded)

### Instantiate models

In [2]:
from tractoencoder_gsoc.ae_model import IncrFeatStridedConvFCUpsampReflectPadAE as AEKeras
model_torch = AETorch(32)
model_keras = AEKeras(32)

2024-06-20 11:35:15.007344: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 11:35:15.044821: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 11:35:15.854026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Compare the shape of the weights and biases in some layers between Torch and Keras to see the difference pattern

In [3]:
print("ENCODER LAYER CONV6")
print(f"Weights TORCH: {model_torch.encod_conv6[1].weight.shape}")
print(f"Biases TORCH: {model_torch.encod_conv6[1].bias.shape}")
print("---------------------------------")
print(f"Weights KERAS: {model_keras.model.get_layer('encoder').encod_conv6.layers[1].get_weights()[0].shape}")
print(f"Biases KERAS: {model_keras.model.get_layer('encoder').encod_conv6.layers[1].get_weights()[1].shape}")

print("\n\nENCODER LAYER CONV3")
print(f"Weights TORCH: {model_torch.encod_conv3[1].weight.shape}")
print(f"Biases TORCH: {model_torch.encod_conv3[1].bias.shape}")
print("---------------------------------")
print(f"Weights KERAS: {model_keras.model.get_layer('encoder').encod_conv3.layers[1].get_weights()[0].shape}")
print(f"Biases KERAS: {model_keras.model.get_layer('encoder').encod_conv3.layers[1].get_weights()[1].shape}")

print("\n\nDECODER LAYER FC1")
print(f"Weights TORCH: {model_torch.fc1.weight.shape}")
print(f"Biases TORCH: {model_torch.fc1.bias.shape}")
print("---------------------------------")
print(f"Weights KERAS: {model_keras.model.get_layer('encoder').fc1.get_weights()[0].shape}")
print(f"Biases KERAS: {model_keras.model.get_layer('encoder').fc1.get_weights()[1].shape}")


print("\n\nDECODER LAYER CONV1")
print(f"Weights TORCH: {model_torch.decod_conv1[1].weight.shape}")
print(f"Biases TORCH: {model_torch.decod_conv1[1].bias.shape}")
print("---------------------------------")
print(f"Weights KERAS: {model_keras.model.get_layer('decoder').decod_conv1.layers[1].get_weights()[0].shape}")
print(f"Biases KERAS: {model_keras.model.get_layer('decoder').decod_conv1.layers[1].get_weights()[1].shape}")



ENCODER LAYER CONV6
Weights TORCH: torch.Size([1024, 512, 3])
Biases TORCH: torch.Size([1024])
---------------------------------
Weights KERAS: (3, 512, 1024)
Biases KERAS: (1024,)


ENCODER LAYER CONV3
Weights TORCH: torch.Size([128, 64, 3])
Biases TORCH: torch.Size([128])
---------------------------------
Weights KERAS: (3, 64, 128)
Biases KERAS: (128,)


DECODER LAYER FC1
Weights TORCH: torch.Size([32, 8192])
Biases TORCH: torch.Size([32])
---------------------------------
Weights KERAS: (8192, 32)
Biases KERAS: (32,)


DECODER LAYER CONV1
Weights TORCH: torch.Size([512, 1024, 3])
Biases TORCH: torch.Size([512])
---------------------------------
Weights KERAS: (3, 1024, 512)
Biases KERAS: (512,)


# READ WEIGHTS FROM PTH FILE

In [4]:
import os
import torch
pth_path = os.path.abspath("/home/teitxe/data/fibercup_model/best_model_fibercup.pt")
torch_weights = torch.load(pth_path, map_location=torch.device('cpu'))

In [5]:
print(torch_weights.keys())
print(torch_weights['epoch'])

dict_keys(['epoch', 'state_dict', 'lowest_loss', 'optimizer'])
120


In [6]:
print(torch_weights['state_dict'].keys())
weight_dict = torch_weights['state_dict']
model_torch.load_state_dict(torch_weights['state_dict'])

odict_keys(['encod_conv1.1.weight', 'encod_conv1.1.bias', 'encod_conv2.1.weight', 'encod_conv2.1.bias', 'encod_conv3.1.weight', 'encod_conv3.1.bias', 'encod_conv4.1.weight', 'encod_conv4.1.bias', 'encod_conv5.1.weight', 'encod_conv5.1.bias', 'encod_conv6.1.weight', 'encod_conv6.1.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'decod_conv1.1.weight', 'decod_conv1.1.bias', 'decod_conv2.1.weight', 'decod_conv2.1.bias', 'decod_conv3.1.weight', 'decod_conv3.1.bias', 'decod_conv4.1.weight', 'decod_conv4.1.bias', 'decod_conv5.1.weight', 'decod_conv5.1.bias', 'decod_conv6.1.weight', 'decod_conv6.1.bias'])


<All keys matched successfully>

## Set the weights in the layers setting the read Pytorch weights to the Keras model

### Encoder

In [7]:
import numpy as np
data_type = np.float32

In [8]:
# encod_conv1
weight_bias_list = [weight_dict['encod_conv1.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['encod_conv1.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('encoder').encod_conv1.layers[1].set_weights(weight_bias_list)

# encod_conv2
weight_bias_list = [weight_dict['encod_conv2.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['encod_conv2.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('encoder').encod_conv2.layers[1].set_weights(weight_bias_list)

# encod_conv3
weight_bias_list = [weight_dict['encod_conv3.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['encod_conv3.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('encoder').encod_conv3.layers[1].set_weights(weight_bias_list)

# encod_conv4
weight_bias_list = [weight_dict['encod_conv4.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['encod_conv4.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('encoder').encod_conv4.layers[1].set_weights(weight_bias_list)

# encod_conv5
weight_bias_list = [weight_dict['encod_conv5.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['encod_conv5.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('encoder').encod_conv5.layers[1].set_weights(weight_bias_list)

# encod_conv6
weight_bias_list = [weight_dict['encod_conv6.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['encod_conv6.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('encoder').encod_conv6.layers[1].set_weights(weight_bias_list)

# fc1
weight_bias_list = [weight_dict['fc1.weight'].numpy().transpose(1, 0).astype(data_type),
               weight_dict['fc1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('encoder').fc1.set_weights(weight_bias_list)


### Decoder

In [9]:
# fc2
weight_bias_list = [weight_dict['fc2.weight'].numpy().transpose(1, 0).astype(data_type),
               weight_dict['fc2.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('decoder').fc2.set_weights(weight_bias_list)

# decod_conv1
weight_bias_list = [weight_dict['decod_conv1.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['decod_conv1.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('decoder').decod_conv1.layers[1].set_weights(weight_bias_list)

# decod_conv2
weight_bias_list = [weight_dict['decod_conv2.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['decod_conv2.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('decoder').decod_conv2.layers[1].set_weights(weight_bias_list)

# decod_conv3
weight_bias_list = [weight_dict['decod_conv3.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['decod_conv3.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('decoder').decod_conv3.layers[1].set_weights(weight_bias_list)

# decod_conv4
weight_bias_list = [weight_dict['decod_conv4.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['decod_conv4.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('decoder').decod_conv4.layers[1].set_weights(weight_bias_list)

# decod_conv5
weight_bias_list = [weight_dict['decod_conv5.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['decod_conv5.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('decoder').decod_conv5.layers[1].set_weights(weight_bias_list)

# decod_conv6
weight_bias_list = [weight_dict['decod_conv6.1.weight'].numpy().transpose(2, 1, 0).astype(data_type),
               weight_dict['decod_conv6.1.bias'].numpy().astype(data_type)]
model_keras.model.get_layer('decoder').decod_conv6.layers[1].set_weights(weight_bias_list)


## Check that the weights are equal in both models

### Encoder

In [10]:
# encod_conv1 weights
print(f"Encod_conv1 weights: {np.all(model_keras.model.get_layer('encoder').encod_conv1.layers[1].get_weights()[0] == model_torch.encod_conv1[1].weight.detach().numpy().transpose(2, 1, 0))}")
# encod_conv1 biases
print(f"Encod_conv1 biases: {np.all(model_keras.model.get_layer('encoder').encod_conv1.layers[1].get_weights()[1] == model_torch.encod_conv1[1].bias.detach().numpy())}")

# encod_conv2 weights
print(f"Encod_conv2 weights: {np.all(model_keras.model.get_layer('encoder').encod_conv2.layers[1].get_weights()[0] == model_torch.encod_conv2[1].weight.detach().numpy().transpose(2, 1, 0))}")
# encod_conv2 biases
print(f"Encod_conv2 biases: {np.all(model_keras.model.get_layer('encoder').encod_conv2.layers[1].get_weights()[1] == model_torch.encod_conv2[1].bias.detach().numpy())}")

# encod_conv3 weights
print(f"Encod_conv3 weights: {np.all(model_keras.model.get_layer('encoder').encod_conv3.layers[1].get_weights()[0] == model_torch.encod_conv3[1].weight.detach().numpy().transpose(2, 1, 0))}")
# encod_conv3 biases
print(f"Encod_conv3 biases: {np.all(model_keras.model.get_layer('encoder').encod_conv3.layers[1].get_weights()[1] == model_torch.encod_conv3[1].bias.detach().numpy())}")

# encod_conv4 weights
print(f"Encod_conv4 weights: {np.all(model_keras.model.get_layer('encoder').encod_conv4.layers[1].get_weights()[0] == model_torch.encod_conv4[1].weight.detach().numpy().transpose(2, 1, 0))}")
# encod_conv4 biases
print(f"Encod_conv4 biases: {np.all(model_keras.model.get_layer('encoder').encod_conv4.layers[1].get_weights()[1] == model_torch.encod_conv4[1].bias.detach().numpy())}")

# encod_conv5 weights
print(f"Encod_conv5 weights: {np.all(model_keras.model.get_layer('encoder').encod_conv5.layers[1].get_weights()[0] == model_torch.encod_conv5[1].weight.detach().numpy().transpose(2, 1, 0))}")
# encod_conv5 biases
print(f"Encod_conv5 biases: {np.all(model_keras.model.get_layer('encoder').encod_conv5.layers[1].get_weights()[1] == model_torch.encod_conv5[1].bias.detach().numpy())}")

# encod_conv6 weights
print(f"Encod_conv6 weights: {np.all(model_keras.model.get_layer('encoder').encod_conv6.layers[1].get_weights()[0] == model_torch.encod_conv6[1].weight.detach().numpy().transpose(2, 1, 0))}")
# encod_conv6 biases
print(f"Encod_conv6 biases: {np.all(model_keras.model.get_layer('encoder').encod_conv6.layers[1].get_weights()[1] == model_torch.encod_conv6[1].bias.detach().numpy())}")

# fc1 weights
print(f"FC1 weights: {np.all(model_keras.model.get_layer('encoder').fc1.get_weights()[0] == model_torch.fc1.weight.detach().numpy().transpose(1, 0))}")
# fc1 biases
print(f"FC1 biases: {np.all(model_keras.model.get_layer('encoder').fc1.get_weights()[1] == model_torch.fc1.bias.detach().numpy())}")

Encod_conv1 weights: True
Encod_conv1 biases: True
Encod_conv2 weights: True
Encod_conv2 biases: True
Encod_conv3 weights: True
Encod_conv3 biases: True
Encod_conv4 weights: True
Encod_conv4 biases: True
Encod_conv5 weights: True
Encod_conv5 biases: True
Encod_conv6 weights: True
Encod_conv6 biases: True
FC1 weights: True
FC1 biases: True


### Decoder

In [11]:
# fc2 weights
print(f"FC2 weights: {np.all(model_keras.model.get_layer('decoder').fc2.get_weights()[0] == model_torch.fc2.weight.detach().numpy().transpose(1, 0))}")
# fc2 biases
print(f"FC2 biases: {np.all(model_keras.model.get_layer('decoder').fc2.get_weights()[1] == model_torch.fc2.bias.detach().numpy())}")

# decod_conv1 weights
print(f"Decod_conv1 weights: {np.all(model_keras.model.get_layer('decoder').decod_conv1.layers[1].get_weights()[0] == model_torch.decod_conv1[1].weight.detach().numpy().transpose(2, 1, 0))}")
# decod_conv1 biases
print(f"Decod_conv1 biases: {np.all(model_keras.model.get_layer('decoder').decod_conv1.layers[1].get_weights()[1] == model_torch.decod_conv1[1].bias.detach().numpy())}")

# decod_conv2 weights
print(f"Decod_conv2 weights: {np.all(model_keras.model.get_layer('decoder').decod_conv2.layers[1].get_weights()[0] == model_torch.decod_conv2[1].weight.detach().numpy().transpose(2, 1, 0))}")
# decod_conv2 biases
print(f"Decod_conv2 biases: {np.all(model_keras.model.get_layer('decoder').decod_conv2.layers[1].get_weights()[1] == model_torch.decod_conv2[1].bias.detach().numpy())}")

# decod_conv3 weights
print(f"Decod_conv3 weights: {np.all(model_keras.model.get_layer('decoder').decod_conv3.layers[1].get_weights()[0] == model_torch.decod_conv3[1].weight.detach().numpy().transpose(2, 1, 0))}")
# decod_conv3 biases
print(f"Decod_conv3 biases: {np.all(model_keras.model.get_layer('decoder').decod_conv3.layers[1].get_weights()[1] == model_torch.decod_conv3[1].bias.detach().numpy())}")

# decod_conv4 weights
print(f"Decod_conv4 weights: {np.all(model_keras.model.get_layer('decoder').decod_conv4.layers[1].get_weights()[0] == model_torch.decod_conv4[1].weight.detach().numpy().transpose(2, 1, 0))}")
# decod_conv4 biases
print(f"Decod_conv4 biases: {np.all(model_keras.model.get_layer('decoder').decod_conv4.layers[1].get_weights()[1] == model_torch.decod_conv4[1].bias.detach().numpy())}")

# decod_conv5 weights
print(f"Decod_conv5 weights: {np.all(model_keras.model.get_layer('decoder').decod_conv5.layers[1].get_weights()[0] == model_torch.decod_conv5[1].weight.detach().numpy().transpose(2, 1, 0))}")
# decod_conv5 biases
print(f"Decod_conv5 biases: {np.all(model_keras.model.get_layer('decoder').decod_conv5.layers[1].get_weights()[1] == model_torch.decod_conv5[1].bias.detach().numpy())}")

# decod_conv6 weights
print(f"Decod_conv6 weights: {np.all(model_keras.model.get_layer('decoder').decod_conv6.layers[1].get_weights()[0] == model_torch.decod_conv6[1].weight.detach().numpy().transpose(2, 1, 0))}")
# decod_conv6 biases
print(f"Decod_conv6 biases: {np.all(model_keras.model.get_layer('decoder').decod_conv6.layers[1].get_weights()[1] == model_torch.decod_conv6[1].bias.detach().numpy())}")

FC2 weights: True
FC2 biases: True
Decod_conv1 weights: True
Decod_conv1 biases: True
Decod_conv2 weights: True
Decod_conv2 biases: True
Decod_conv3 weights: True
Decod_conv3 biases: True
Decod_conv4 weights: True
Decod_conv4 biases: True
Decod_conv5 weights: True
Decod_conv5 biases: True
Decod_conv6 weights: True
Decod_conv6 biases: True


### Check that both models give a very similar output for the same input

In [14]:
import numpy as np
import tensorflow as tf

from tractoencoder_gsoc import utils
trk = os.path.abspath("../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/train/fibercup_Simulated_prob_tracking_minL10_resampled256_plausibles_std_endpoints_train.trk")
anat = os.path.abspath("../data/fibercup/Simulated_FiberCup.nii.gz")
random_streamlines = utils.prepare_tensor_from_file(trk, anat).numpy()
dummy_input_keras = tf.convert_to_tensor(random_streamlines, dtype=tf.float32)
dummy_input_torch = torch.Tensor(random_streamlines.transpose(0, 2, 1))

output_keras = model_keras(dummy_input_keras)
output_torch = model_torch(dummy_input_torch).detach().numpy()
output_torch_reshaped = output_torch.transpose(0, 2, 1)

print("MSE: ", np.mean((output_keras - output_torch_reshaped) ** 2))
print("diff max (3D): ", np.max(output_keras - output_torch_reshaped))
print("diff norm (3D): ", np.linalg.norm(output_keras - output_torch_reshaped))
are_close = np.isclose(output_keras, output_torch_reshaped, atol=1e-6)

# Check if all elements are close
outputs_are_similar = np.all(are_close)
print("Outputs are similar: ", outputs_are_similar)

# Save the output tractograms
data_wd = os.path.abspath("./data_outputs/model_comparison/")
os.makedirs(data_wd, exist_ok=True)
output_fname_keras = os.path.join(data_wd,
                                  "output_keras.trk")
utils.save_tractogram(output_keras, output_fname_keras)
output_fname_torch = os.path.join(data_wd,
                                  "output_torch.trk")
utils.save_tractogram(output_torch_reshaped, output_fname_torch)

MSE:  4637.233
diff max (3D):  162.8513
diff norm (3D):  105273.695
Outputs are similar:  False


### Outputs are not similar. Let's check where this happens in the models.

Extract the layers

In [15]:
# encoder Keras
encoder = model_keras.model.get_layer('encoder')
encoder_keras_layers = [getattr(encoder, layer) for layer in dir(encoder) if layer.startswith(('encod_conv', 'fc'))]
# encoder Torch
encoder_torch_layers = [getattr(model_torch, layer) for layer in dir(model_torch) if layer.startswith(('encod_conv', 'fc1'))]

# decoder Keras
decoder = model_keras.model.get_layer('decoder')
decoder_keras_layers = [getattr(decoder, layer) for layer in dir(decoder) if layer.startswith(('decod_conv', 'fc'))]
# decoder Torch
decoder_torch_layers = [getattr(model_torch, layer) for layer in dir(model_torch) if layer.startswith(('decod_conv', 'fc2'))]

Iterate through the conv1d layers and check if the output is similar.

In [19]:
import numpy as np
import tensorflow as tf

random_streamlines = utils.prepare_tensor_from_file(trk, anat).numpy()
dummy_input_keras = tf.convert_to_tensor(random_streamlines, dtype=tf.float32)
dummy_input_torch = torch.Tensor(random_streamlines.transpose(0, 2, 1))

for i, (layer_keras, layer_torch) in enumerate(zip(encoder_keras_layers[:-1], encoder_torch_layers[:-1])):
    print(f"ENCODER LAYER {i+1}")
    print(f"Keras: {layer_keras}")
    print(f"Torch: {layer_torch}")
    # Run the input through the layers
    layer_output_keras = dummy_input_keras
    layer_output_torch = dummy_input_torch
    for c in range(i+1):
        if c == 5:  # conv6, do not ReLU
            layer_output_keras = encoder_keras_layers[c](layer_output_keras)
            layer_output_torch = encoder_torch_layers[c](layer_output_torch)
        else:
            layer_output_keras = tf.nn.relu(encoder_keras_layers[c](layer_output_keras))
            layer_output_torch = F.relu(encoder_torch_layers[c](layer_output_torch))
    # Check if the outputs are similar
    layer_output_torch_reshaped = layer_output_torch.detach().numpy().transpose(0, 2, 1)
    are_close = np.all(np.isclose(layer_output_keras, layer_output_torch_reshaped, atol=1e-6))
    print(f"In layer {i+1} Outputs are similar? {str(are_close).upper()}")
    print(f"MAE is: {np.mean(np.abs(layer_output_keras - layer_output_torch_reshaped))}")
    print(f"MSE is: {np.mean((layer_output_keras - layer_output_torch_reshaped) ** 2)}\n")


# reshape before running into fc1. Match PyTorch's behavior by transposing the output first
encoder_out_size_keras = (layer_output_keras.shape[1], layer_output_keras.shape[2])
h7_keras = tf.transpose(layer_output_keras, perm=[0, 2, 1])
h7_keras = tf.keras.layers.Flatten()(h7_keras)

encoder_out_size_torch = (layer_output_torch.shape[1], layer_output_torch.shape[2])
h7_torch = layer_output_torch.view(-1, encoder_out_size_torch[0] * encoder_out_size_torch[1]).detach().numpy()

print("ENCODER LAYER FLATTEN")
are_close = np.all(np.isclose(h7_keras, h7_torch, atol=1e-6))
print(f"Keras: Reshaping (flattening before fc1)")
print(f"Torch: Reshaping (flattening before fc1)")
print(f"In layer FLATTEN Outputs are similar? {str(are_close).upper()}")
print(f"MAE is: {np.mean(np.abs(h7_keras - h7_torch))}")
print(f"MSE is: {np.mean((h7_keras - h7_torch) ** 2)}\n")

# fc1
print("ENCODER LAYER FC1")
fc1_keras = tf.nn.relu(encoder_keras_layers[-1](h7_keras))
fc1_torch = F.relu(encoder_torch_layers[-1](torch.Tensor(h7_torch))).detach().numpy()
are_close = np.all(np.isclose(h7_keras, h7_torch, atol=1e-6))
print(f"Keras: {encoder_keras_layers[-1]}")
print(f"Torch: {encoder_torch_layers[-1]}")
print(f"In layer {7} Outputs are similar? {str(are_close).upper()}")
print(f"MAE is: {np.mean(np.abs(fc1_keras - fc1_torch))}")
print(f"MSE is: {np.mean((fc1_keras - fc1_torch) ** 2)}\n")


ENCODER LAYER 1
Keras: <Sequential name=sequential, built=True>
Torch: Sequential(
  (0): ReflectionPad1d((1, 1))
  (1): Conv1d(3, 32, kernel_size=(3,), stride=(2,))
)
In layer 1 Outputs are similar? FALSE
MAE is: 3.2603165323052963e-07
MSE is: 1.1156285313965175e-12

ENCODER LAYER 2
Keras: <Sequential name=sequential_1, built=True>
Torch: Sequential(
  (0): ReflectionPad1d((1, 1))
  (1): Conv1d(32, 64, kernel_size=(3,), stride=(2,))
)
In layer 2 Outputs are similar? FALSE
MAE is: 1.8141469126931042e-07
MSE is: 3.846853120336585e-13

ENCODER LAYER 3
Keras: <Sequential name=sequential_2, built=True>
Torch: Sequential(
  (0): ReflectionPad1d((1, 1))
  (1): Conv1d(64, 128, kernel_size=(3,), stride=(2,))
)
In layer 3 Outputs are similar? FALSE
MAE is: 3.93934698195153e-08
MSE is: 6.669323974377653e-14

ENCODER LAYER 4
Keras: <Sequential name=sequential_3, built=True>
Torch: Sequential(
  (0): ReflectionPad1d((1, 1))
  (1): Conv1d(128, 256, kernel_size=(3,), stride=(2,))
)
In layer 4 Output

## OUTPUTS ARE (SIMILAR)? IN THE ENCODER. LET'S CHECK THE DECODER

In [21]:
# TODO: CHECK THIS CODE AGAIN/ MAYBE WRITE SEQUENTIALLY AND NOT LOOP
# Decoder (fc1 is the input to it)

print("DECODER LAYER FC2")
fc2_keras = tf.nn.relu(decoder_keras_layers[-1](fc1_keras))
fc2_torch = F.relu(decoder_torch_layers[-1](torch.Tensor(fc1_torch))).detach().numpy()
are_close = np.all(np.isclose(fc2_keras, fc2_torch, atol=1e-6))
print(f"Keras: {decoder_keras_layers[-1]}")
print(f"Torch: {decoder_torch_layers[-1]}")
print(f"In layer FC2 Outputs are similar? {str(are_close).upper()}")
print(f"MAE: {np.mean(np.abs(layer_output_keras - layer_output_torch_reshaped))}")
print(f"MSE is: {np.mean((fc2_keras - fc2_torch) ** 2)}\n")

# Reshape before running into the first conv layer
# Mimic PyTorch behavior by reshaping the output first, then transpose it for our weights to work
fc2_keras_t = tf.reshape(fc2_keras, (-1, encoder_out_size_keras[1], encoder_out_size_keras[0]))
fc2_keras = tf.transpose(fc2_keras_t, perm=[0, 2, 1])
fc2_torch = torch.Tensor(fc2_torch).view(-1,
                                         encoder_out_size_torch[0],
                                         encoder_out_size_torch[1])

for i, (layer_keras, layer_torch) in enumerate(zip(decoder_keras_layers[:-1], decoder_torch_layers[:-1])):
    print(f"ENCODER LAYER {i+1}")
    print(f"Keras: {layer_keras}")
    print(f"Torch: {layer_torch}")
    # Run the input through the layers
    layer_output_keras = fc2_keras
    layer_output_torch = fc2_torch
    for c in range(i+1):
        layer_output_keras = tf.nn.relu(decoder_keras_layers[c](layer_output_keras))
        layer_output_torch = F.relu(decoder_torch_layers[c](layer_output_torch))
    # Check if the outputs are similar
    layer_output_torch_reshaped = np.transpose(layer_output_torch.detach().numpy(), (0, 2, 1))
    are_close = np.all(np.isclose(layer_output_keras, layer_output_torch_reshaped, atol=1e-6))
    print(f"In layer {i+1} Outputs are similar? {str(are_close).upper()}")
    print(f"MAE is: {np.mean(np.abs(layer_output_keras - layer_output_torch_reshaped))}")
    print(f"MSE is: {np.mean((layer_output_keras - layer_output_torch_reshaped) ** 2)}\n")

DECODER LAYER FC2
Keras: <Dense name=fc2, built=True>
Torch: Linear(in_features=32, out_features=8192, bias=True)
In layer FC2 Outputs are similar? FALSE
MAE: 3.006216786616278e-07
MSE is: 6.112635137639488e-12

ENCODER LAYER 1
Keras: <Sequential name=sequential_6, built=True>
Torch: Sequential(
  (0): ReflectionPad1d((1, 1))
  (1): Conv1d(1024, 512, kernel_size=(3,), stride=(1,))
)
In layer 1 Outputs are similar? FALSE
MAE is: 2.4263886189146433e-06
MSE is: 1.6551117421048644e-10

ENCODER LAYER 2
Keras: <Sequential name=sequential_7, built=True>
Torch: Sequential(
  (0): ReflectionPad1d((1, 1))
  (1): Conv1d(512, 256, kernel_size=(3,), stride=(1,))
)
In layer 2 Outputs are similar? FALSE
MAE is: 3.7567951949313283e-06
MSE is: 2.109021846052883e-10

ENCODER LAYER 3
Keras: <Sequential name=sequential_8, built=True>
Torch: Sequential(
  (0): ReflectionPad1d((1, 1))
  (1): Conv1d(256, 128, kernel_size=(3,), stride=(1,))
)
In layer 3 Outputs are similar? FALSE
MAE is: 3.276526513218414e-06

=====================

### Put some tractograms into the Models, check the output similarity visually

In [ ]:
data_dir = "../data/fibercup/fibercup_advanced_filtering_no_ushapes/ae_input_std_endpoints/"

f = "fibercup_Simulated_prob_tracking_minL10_resampled256_plausibles_std_endpoints_"
anat = os.path.abspath("../data/fibercup/Simulated_FiberCup.nii.gz")
tractogram_paths = [os.path.abspath(os.path.join(data_dir, "train", f + "train.trk")),
                    os.path.abspath(os.path.join(data_dir, "test", f + "test.trk")),
                    os.path.abspath(os.path.join(data_dir, "valid", f + "valid.trk"))]

In [ ]:
from tractoencoder_gsoc import utils

data_wd = os.path.abspath("./data_outputs/")
os.makedirs(data_wd, exist_ok=True)

for trk_path in tractogram_paths:
    # Load the tractogram
    input_data_keras = utils.prepare_tensor_from_file(trk_path, anat)
    output_keras = model_keras(input_data_keras)
    
    input_data_torch = torch.Tensor(input_data_keras.numpy().transpose(0, 2, 1))
    output_torch = model_torch(input_data_torch)
    
    output_path_torch = os.path.join(data_wd, os.path.basename(trk_path).replace(".trk", "_torch_output.trk"))
    output_path_keras = os.path.join(data_wd, os.path.basename(trk_path).replace(".trk", "_keras_output.trk"))
    utils.save_tractogram(streamlines=output_keras,
                          img_fname=anat,
                          tractogram_fname=output_path_keras)
    utils.save_tractogram(streamlines=output_keras,
                          img_fname=anat,
                          tractogram_fname=output_path_torch)

# EXPERIMENT: Encode a tractogram with the Keras model, decode it with both models

In [ ]:
data_wd = os.path.abspath("./data_outputs/encode_keras_decode_both")
os.makedirs(data_wd, exist_ok=True)

for trk in tractogram_paths[0:1]:
    
    # Take a tractogram from FiberCup
    input_data_keras = utils.prepare_tensor_from_file(trk,
                                                      anat)

    # Run the input through the Keras encoder. Get the latent space representation of the input
    output_encoder_keras = model_keras.model.get_layer('encoder')(input_data_keras)
    # Also run the input through the PyTorch encoder. Compare them just in case
    output_encoder_torch = model_torch.encode(torch.Tensor(input_data_keras.numpy().transpose(0, 2, 1)))
    output_encoder_torch = output_encoder_torch.detach().numpy()
    # Compare output_encoder_keras and output_encoder_torch
    print("Encoder MSE: ", np.mean((output_encoder_keras - output_encoder_torch) ** 2))
    print("Encoder MAE: ", np.mean(np.abs(output_encoder_keras - output_encoder_torch)))
    print("Encoder std: ", np.std(output_encoder_keras - output_encoder_torch), "\n")
    # Decode the output of the encoder using the Keras model
    output_model_keras = model_keras.model.get_layer('decoder')(output_encoder_keras)

    # Decode the output of the encoder using the PyTorch model
    output_model_torch = model_torch.decode(torch.Tensor(output_encoder_keras.numpy()))
    output_model_torch = output_model_torch.detach().numpy().transpose(0, 2, 1)

    # Check the difference between the outputs
    print("MSE: ", np.mean((output_model_keras - output_model_torch) ** 2))
    print("MAE: ", np.mean(np.abs(output_model_keras - output_model_torch)))
    print("std: ", np.std(output_model_keras - output_model_torch), "\n")
    
    output_path_torch = os.path.join(data_wd, os.path.basename(trk).replace(".trk", "_torch_output.trk"))
    output_path_keras = os.path.join(data_wd, os.path.basename(trk).replace(".trk", "_keras_output.trk"))

    # Save the output of the Keras model
    utils.save_tractogram(streamlines=output_model_keras.numpy().transpose(0, 2, 1),
                          img_fname=anat,
                          tractogram_fname=output_path_keras)

    # Save the output of the Torch model
    utils.save_tractogram(streamlines=output_model_torch,
                          img_fname=anat,
                          tractogram_fname=output_path_torch)

## Very strange thing happening here.

### If we TORCH(ENCODE-DECODE) -> Bad reconstruction
### If we KERAS(ENCODE-DECODE) -> Bad reconstruction (same as above)
### If we KERAS(ENCODE) and TORCH(DECODE) -> Good reconstruction

# WTH
